In [1]:
from __future__ import division
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%precision 4
plt.style.use('ggplot')
import pystan
import scipy.stats as stats
import numpy as np


plt.rcParams["figure.figsize"] = (12,8)

# Parte 0 - gerando dados artificiais 

In [22]:
g=9.81
pho=1
eta=1
n = 100
a1 = 5
b1 = 5
a2 = 10
b2 = 10
sigma_h1=1
sigma_h2=1
sigma_pg=1e-2
u = np.linspace(0, 10, n)
x = np.linspace(30, 60, n)
h1=a1*x+b1+np.random.randn(n)
h2=a2*u+b2+np.random.randn(n)
q=x
k=g*pho*eta
pg = np.random.normal(g*pho*eta*(h1-h2),sigma_pg)

In [23]:
# o b não pode ser negativo e normalmente é bem alto (é uma altura)
# o a é baixo

#vazão turbinada = q(t) = u - vazao_vertida
#Volume Útil (%) = x 
#Defluência (m³/s) = u 

dados_df=pd.read_csv('IRAPE.csv')

dados_df.drop(len(dados_df)-1,inplace=True)
dados_df.drop(len(dados_df)-1,inplace=True)

In [31]:
warmup=1000
thin=100
chains=4
num_samples=1000
num_iter=num_samples*thin+warmup


u=np.array(dados_df['Defluência (m³/s)'])
x=np.array(dados_df['Volume Útil (%)'])
q=np.array(dados_df['Vazão Turbinada (m³/s)'])
h1=np.array(dados_df['Cota (m)'])
h2=np.array(dados_df['h2'])
pg=np.array(dados_df['PG'])
g=9.81
pho=1000
eta_g=0.95
eta_t=0.92
pc=0.01
k=g*pho*eta_t*eta_g*(1-pc)/1000000
n=len(u)

#dados normalizados
norm_u=(u-u.mean())/u.std()
norm_x=(x-x.mean())/x.std()
norm_pg=(pg-pg.mean())/pg.std()
norm_q=(q-q.mean())/q.std()
norm_h1=(h1-h1.mean())/h1.std()
norm_h2=(h2-h2.mean())/h2.std()

# Parte 1 - comparando modelos

## Modelo 1: h1 e h2 são variaveis normais - versão normalizada 




In [32]:

ger_model_code = """
data {
    int<lower=0> n; 
    real k;
    real x[n];
    real u[n];
    real q[n];
    real pg[n];
    real h1[n];
    real h2[n];
}
transformed data {}
parameters {
    real a1;
    real a2;
    real b1;
    real b2;
    real<lower=0> sigma1;
    real<lower=0> sigma2;
    real<lower=0> sigmag;
}
transformed parameters {
    real mu_pg[n];
    real mu_h1[n];
    real mu_h2[n];
    for (i in 1:n) {
        mu_h1[i] = a1*x[i]+b1;
    }
        
    for (i in 1:n) {
        mu_h2[i] = a2*u[i]+b2;
    }
    for (i in 1:n) {
        mu_pg[i] = k*(mu_h1[i]-mu_h2[i])*q[i];
    }
}
model {
    a1 ~ normal(0,1);
    a2 ~ normal(0,1);
    b1 ~ normal(0,1);
    b2 ~ normal(0,1);
    sigma1 ~ exponential(1);
    sigma2 ~ exponential(1);
    sigmag ~ exponential(1);
    h1 ~ normal(mu_h1,sigma1);
    h2 ~ normal(mu_h2,sigma2);
    pg ~ normal(mu_pg,sigmag);
}
generated quantities {}
"""

ger_model_data = {
             'n': n,
             'k': k,
             'x': norm_x,
             'u': norm_u,
             'q': norm_q,
             'pg': norm_pg,
             'h1':norm_h1,
             'h2':norm_h2
            }

fit_1 = pystan.stan(model_code=ger_model_code, data=ger_model_data,chains=chains,warmup=warmup,thin=thin,iter=num_iter)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_b47d460f7921632389b1a4450341b32f NOW.
In file included from /home/vitorhpmelo/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1944,
                 from /home/vitorhpmelo/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/vitorhpmelo/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /tmp/pystan_pskz72g4/stanfit4anon_model_b47d460f7921632389b1a4450341b32f_7699508937138900262.cpp:709:
/home/vitorhpmelo/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
In file included from /home/vitorhpmelo/anaconda3/lib/python3.9/site-packages/pystan/stan/lib/s


Gradient evaluation took 7.7e-05 seconds
1000 transitions using 10 leapfrog steps per transition would take 0.77 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 7.5e-05 seconds
1000 transitions using 10 leapfrog steps per transition would take 0.75 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 8e-05 seconds
1000 transitions using 10 leapfrog steps per transition would take 0.8 seconds.
Adjust your expectations accordingly!


Iteration:      1 / 101000 [  0%]  (Warmup)
Iteration:      1 / 101000 [  0%]  (Warmup)
Iteration:      1 / 101000 [  0%]  (Warmup)

Gradient evaluation took 8.8e-05 seconds
1000 transitions using 10 leapfrog steps per transition would take 0.88 seconds.
Adjust your expectations accordingly!


Iteration:      1 / 101000 [  0%]  (Warmup)


Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'unknown file name' at line 45)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'unknown file name' at line 45)
Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:

Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'unknown file name' at line 47)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampl

Iteration:   1001 / 101000 [  0%]  (Sampling)
Iteration:   1001 / 101000 [  0%]  (Sampling)
Iteration:   1001 / 101000 [  0%]  (Sampling)
Iteration:   1001 / 101000 [  0%]  (Sampling)
Iteration:  11100 / 101000 [ 10%]  (Sampling)
Iteration:  11100 / 101000 [ 10%]  (Sampling)
Iteration:  11100 / 101000 [ 10%]  (Sampling)
Iteration:  11100 / 101000 [ 10%]  (Sampling)
Iteration:  21200 / 101000 [ 20%]  (Sampling)
Iteration:  21200 / 101000 [ 20%]  (Sampling)
Iteration:  21200 / 101000 [ 20%]  (Sampling)
Iteration:  21200 / 101000 [ 20%]  (Sampling)
Iteration:  31300 / 101000 [ 30%]  (Sampling)
Iteration:  31300 / 101000 [ 30%]  (Sampling)
Iteration:  31300 / 101000 [ 30%]  (Sampling)
Iteration:  31300 / 101000 [ 30%]  (Sampling)
Iteration:  41400 / 101000 [ 40%]  (Sampling)
Iteration:  41400 / 101000 [ 40%]  (Sampling)
Iteration:  41400 / 101000 [ 40%]  (Sampling)
Iteration:  41400 / 101000 [ 40%]  (Sampling)
Iteration:  51500 / 101000 [ 50%]  (Sampling)
Iteration:  51500 / 101000 [ 50%] 

In [34]:
fit_1


For the full summary use 'print(fit)'

Inference for Stan model: anon_model_b47d460f7921632389b1a4450341b32f.
4 chains, each with iter=101000; warmup=1000; thin=100; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

             mean se_mean     sd    2.5%     25%     50%     75%    97.5%  n_eff   Rhat
a1            1.0  3.4e-5 2.1e-3     1.0     1.0     1.0     1.0      1.0   3769    1.0
a2          -0.23  8.8e-4   0.05   -0.34   -0.27   -0.23    -0.2    -0.12   3885    1.0
b1         9.3e-6  3.3e-5 2.1e-3 -4.1e-3 -1.4e-3  2.3e-5  1.4e-3   4.0e-3   3897    1.0
b2        -8.7e-3  8.7e-4   0.05   -0.11   -0.05 -9.1e-3    0.03      0.1   3917    1.0
sigma1       0.04  2.5e-5 1.5e-3    0.03    0.04    0.04    0.04     0.04   3678    1.0
sigma2       0.98  6.3e-4   0.04    0.91    0.95    0.98    1.01     1.06   4016    1.0
sigmag        1.0  6.1e-4   0.04    0.92    0.97     1.0    1.02     1.08   4155    1.0
mu_pg[1]     0.01  6.3e-6 3.9e-4    0.01    0.01    0.01    0.0

## Modelo 2: h1 e h2 são variaveis normais - versão não normalizada 

In [33]:

ger_model_code = """
data {
    int<lower=0> n; 
    real k;
    real x[n];
    real u[n];
    real q[n];
    real pg[n];
    real h1[n];
    real h2[n];
}
transformed data {}
parameters {
    real a1;
    real a2;
    real b1;
    real b2;
    real<lower=0> sigma1;
    real<lower=0> sigma2;
    real<lower=0> sigmag;
}
transformed parameters {
    real mu_pg[n];
    real mu_h1[n];
    real mu_h2[n];
    for (i in 1:n) {
        mu_h1[i] = a1*x[i]+b1;
    }
        
    for (i in 1:n) {
        mu_h2[i] = a2*u[i]+b2;
    }
    for (i in 1:n) {
        mu_pg[i] = k*(mu_h1[i]-mu_h2[i])*q[i];
    }
}
model {
    a1 ~ normal(0,100);
    a2 ~ normal(0,100);
    b1 ~ uniform(0,1000);
    b2 ~ uniform(0,1000);
    sigma1 ~ exponential(1);
    sigma2 ~ exponential(1);
    sigmag ~ exponential(1);
    h1 ~ normal(mu_h1,sigma1);
    h2 ~ normal(mu_h2,sigma2);
    pg ~ normal(mu_pg,sigmag);
}
generated quantities {}
"""

ger_model_data = {
             'n': n,
             'k': k,
             'x': x,
             'u': u,
             'q': q,
             'pg': pg,
             'h1':h1,
             'h2':h2
            }


fit_2 = pystan.stan(model_code=ger_model_code, data=ger_model_data,chains=chains,warmup=warmup,thin=thin,iter=num_iter)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_86fc9ad618d14de949f8fc33d56b3890 NOW.
In file included from /home/vitorhpmelo/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1944,
                 from /home/vitorhpmelo/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/vitorhpmelo/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /tmp/pystan_j3ibozz8/stanfit4anon_model_86fc9ad618d14de949f8fc33d56b3890_6279832575903164010.cpp:709:
/home/vitorhpmelo/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
In file included from /home/vitorhpmelo/anaconda3/lib/python3.9/site-packages/pystan/stan/lib/s

Rejecting initial value:
  Log probability evaluates to log(0), i.e. negative infinity.
  Stan can't start sampling from this initial value.
Rejecting initial value:
  Log probability evaluates to log(0), i.e. negative infinity.
  Stan can't start sampling from this initial value.
Rejecting initial value:
  Log probability evaluates to log(0), i.e. negative infinity.
  Stan can't start sampling from this initial value.
Rejecting initial value:Rejecting initial value:

  Log probability evaluates to log(0), i.e. negative infinity.  Log probability evaluates to log(0), i.e. negative infinity.

  Stan can't start sampling from this initial value.  Stan can't start sampling from this initial value.

Rejecting initial value:
  Log probability evaluates to log(0), i.e. negative infinity.
  Stan can't start sampling from this initial value.

Gradient evaluation took 8.5e-05 seconds
1000 transitions using 10 leapfrog steps per transition would take 0.85 seconds.
Adjust your expectations accord

In [35]:
fit_2


For the full summary use 'print(fit)'

Inference for Stan model: anon_model_86fc9ad618d14de949f8fc33d56b3890.
4 chains, each with iter=101000; warmup=1000; thin=100; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

            mean se_mean     sd   2.5%    25%    50%    75%   97.5%  n_eff   Rhat
a1          0.31  1.0e-5 6.6e-4   0.31   0.31   0.31   0.31    0.32   4037    1.0
a2         -0.01  4.2e-5 2.7e-3  -0.02  -0.02  -0.01  -0.01 -8.4e-3   3980    1.0
b1        478.89  8.3e-4   0.05 478.79 478.85 478.89 478.92  478.99   4012    1.0
b2        333.51  9.2e-3   0.58 332.37 333.12 333.51 333.89  334.64   3977    1.0
sigma1      0.15  9.4e-5 6.0e-3   0.14   0.15   0.15   0.16    0.17   4082    1.0
sigma2      9.63  6.0e-3   0.37   8.94   9.38   9.62   9.87   10.41   3812    1.0
sigmag      7.36  4.6e-3    0.3    6.8   7.16   7.36   7.56    7.97   4034    1.0
mu_pg[1]   70.78  3.3e-3   0.21  70.39  70.65  70.78  70.92    71.2   3965    1.0
mu_pg[2]    71.1  3.3e-3   0.

## Modelo 3: h1, h2  e q são implicitos (não são variaveis aleatorias) - desnormalizado

In [ ]:

ger_model_code = """
data {
    int<lower=0> n; 
    real k;
    real x[n];
    real u[n];
    real q[n];
    real pg[n];
}
transformed data {}
parameters {
    real a1;
    real a2;
    real b1;
    real b2;
    real<lower=0> sigmag;
}
transformed parameters {
    real mu_pg[n];
    for (i in 1:n) {
        mu_pg[i] = k*(a1*x[i]+b1-(a2*u[i]+b2))*q[i];
    }
}
model {
    a1 ~ normal(0,100);
    a2 ~ normal(0,100);
    b1 ~ normal(0,100);
    b2 ~ normal(0,100);
    sigmag ~ exponential(1);
    pg ~ normal(mu_pg,sigmag);
}
generated quantities {}
"""

ger_model_data = {
             'n': n,
             'k': k,
             'x': x,
             'u': u,
             'q': q,
             'pg': pg,

            }


fit_3 = pystan.stan(model_code=ger_model_code, data=ger_model_data,chains=chains,warmup=warmup,thin=thin,iter=num_iter)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_aabcdb3cd28fb59acf6614f3cce4076e NOW.
C:\Users\henri\anaconda3\envs\lps\lib\site-packages\Cython\Compiler\Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: C:\Users\henri\AppData\Local\Temp\tmpm_1756yf\stanfit4anon_model_aabcdb3cd28fb59acf6614f3cce4076e_6153808054124000769.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [ ]:
fit_3

## Modelo 4: h1, h2 e q são implicitos (não são variaveis aleatorias) - normalizado

In [ ]:

ger_model_code = """
data {
    int<lower=0> n; 
    real k;
    real x[n];
    real u[n];
    real q[n];
    real pg[n];
}
transformed data {}
parameters {
    real a1;
    real a2;
    real b1;
    real b2;
    real<lower=0> sigmag;
}
transformed parameters {
    real mu_pg[n];
    
    for (i in 1:n) {
        mu_pg[i] = k*(a1*x[i]+b1-(a2*u[i]+b2))*q[i];
    }
}
model {
    a1 ~ normal(0,1);
    a2 ~ normal(0,1);
    b1 ~ normal(0,1);
    b2 ~ normal(0,1);
    sigmag ~ inv_gamma(0.001,0.001);
    pg ~ normal(mu_pg,sigmag);
}
generated quantities {}
"""

ger_model_data = {
             'n': n,
             'k': k,
             'x': norm_x,
             'u': norm_u,
             'q': norm_q,
             'pg': norm_pg,
            }


fit_4 = pystan.stan(model_code=ger_model_code, data=ger_model_data,chains=chains,warmup=warmup,thin=thin,iter=num_iter)

In [ ]:
fit_4

# Parte 2 - fazendo análises do modelo em si

In [ ]:

fit_4.plot(['a1', 'a2','b1','b2']);
plt.tight_layout()

# Parte 3 - Fazendo a predição dado um valor de [x,u]


In [ ]:
error=[]
a1_post=fit_4['a1']
a2_post=fit_4['a2']
b1_post=fit_4['b1']
b2_post=fit_4['b2']
sigmag=fit_4['sigmag']
for i in range(n):
    x_pred=norm_x[i]
    u_pred=norm_q[i]
    q_pred=norm_u[i]
    h1_post=a1_post*x_pred+b1_post
    h2_post=a2_post*u_pred+b2_post

    post_samples=np.random.normal(k*(h1_post-h2_post)*q_pred,sigmag)
    error.append(100*np.abs((post_samples.mean()-norm_pg[i])/norm_pg[i]))

In [ ]:
sum(error)/len(error)